# ASSESSMENT 1

## Implement the Naive Bayes and K-Nearest Neighbours algorithms from scratch in Python using Numpy and Pandas and Matplotlib for visualization.

## The algorithm must be implemented as a function with arguments, x_train (the features) and y_train (the output).  Usage of any library that has an implementation is forbidden.

## The code must be uploaded to the portal and GitHub and be in Python Notebook format (.ipynb file). The GitHub link is to be attached. The repository has to be open.

## Evaluation will be done on the code formatting, the final accuracy results, and small explanations of any data preprocessing done (data preprocessing and visualization has bonus points). The explanations must also be included in the same python notebook which can be done using markdown cells.

### Reading the dataset

In [1]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
df = pd.concat([train, test]) 
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [2]:
df.shape

(1309, 12)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [4]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,1309.000000,891.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000
mean,655.000000,0.383838,2.294882,29.881138,0.498854,0.385027,33.295479
std,378.020061,0.486592,0.837836,14.413493,1.041658,0.865560,51.758668
min,1.000000,0.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,328.000000,0.000000,2.000000,21.000000,0.000000,0.000000,7.895800
50%,655.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,982.000000,1.000000,3.000000,39.000000,1.000000,0.000000,31.275000
max,1309.000000,1.000000,3.000000,80.000000,8.000000,9.000000,512.329200


In [5]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [6]:
df.isna().sum()

PassengerId       0
Survived        418
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
dtype: int64

### Preprocessing the Data

In [7]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])

In [9]:
# Encoding the categorical data

df['Sex'] = df['Sex'].map({
    'male': 0, 
    'female': 1
})

In [10]:
df['Embarked'] = df['Embarked'].map({
    'S': 0, 
    'C': 1, 
    'Q': 2
})

In [14]:
# Feature Selection
df = df[['Sex', 'Age', 'SibSp', 'Parch', 'Pclass', 'Fare' , 'Embarked','Survived']]
corr = df.corr()['Survived']
print(corr.sort_values(ascending=False))

Survived    1.000000
Sex         0.543351
Fare        0.257307
Embarked    0.106811
Parch       0.081629
SibSp      -0.035322
Age        -0.070323
Pclass     -0.338481
Name: Survived, dtype: float64


In [15]:
df['Family'] = df['SibSp'] + df['Parch'] + 1 

/var/folders/r4/zrgy00s550x2y02w1lgxv5mc0000gn/T/ipykernel_56733/2882356983.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Family'] = df['SibSp'] + df['Parch'] + 1


In [16]:
corr = df.corr()['Survived']
print(corr.sort_values(ascending=False))

Survived    1.000000
Sex         0.543351
Fare        0.257307
Embarked    0.106811
Parch       0.081629
Family      0.016639
SibSp      -0.035322
Age        -0.070323
Pclass     -0.338481
Name: Survived, dtype: float64


In [17]:
# Splitting the dataset
train_clean = df[:len(train)]
test_clean = df[len(train):]

In [19]:
target = 'Survived'
features = ['Sex','Fare','Embarked','SibSp','Age']

X_train = train_clean[features].values
y_train = train_clean[target].values
X_test = test_clean[features].values

In [20]:
from sklearn.model_selection import train_test_split

x_train_split, x_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Designing the Naive bayes Algorithm

In [22]:
def gauss_prob(x, mean, var):
    eps = 1e-6  # avoid division by zero
    coeff = 1.0 / np.sqrt(2.0 * np.pi * var + eps)
    exponent = np.exp(-((x - mean) ** 2) / (2 * var + eps))
    return coeff * exponent

def fit_naive_bayes(X, y):
    classes = np.unique(y)
    means = {}
    vars_ = {}
    priors = {}

    for c in classes:
        Xc = X[y == c]
        means[c] = np.mean(Xc, axis=0)
        vars_[c] = np.var(Xc, axis=0)
        priors[c] = len(Xc) / len(X)
    
    return classes, means, vars_, priors

def predict_naive_bayes(X, classes, means, vars_, priors):
    preds = []
    for x in X:
        posteriors = []
        for c in classes:
            prior = np.log(priors[c])
            likelihood = np.sum(np.log(gauss_prob(x, means[c], vars_[c])))
            posteriors.append(prior + likelihood)
        preds.append(classes[np.argmax(posteriors)])
    return np.array(preds)

### Designing the k Nearest Neighbors

### Designing the k-Nearest Neighbors 

In [24]:
def fit_knn(X_train, y_train):
    return X_train, y_train

def predict_knn(X_train, y_train, X_test, k=3):
    preds = []
    for x in X_test:
        dists = np.linalg.norm(X_train - x, axis=1)
        idx = np.argsort(dists)[:k]
        k_labels = y_train[idx]
        label = Counter(k_labels).most_common(1)[0][0]
        preds.append(label)
    return np.array(preds)

### Training Naive Bayes Model

In [29]:
import numpy as np

cls, means, vars_, priors = fit_naive_bayes(x_train, y_train)
nb_preds = predict_naive_bayes(x_test, cls, means, vars_, priors)

### Training k-Nearest Neighbors

In [28]:
from collections import Counter

X_train_knn, y_train_knn = fit_knn(x_train, y_train)
knn_preds_val = predict_knn(X_train_knn, y_train_knn, x_val, k=5)

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Naive Bayes
nb_preds = predict_naive_bayes(x_val, cls, means, vars_, priors)
print("Naive Bayes Validation Performance:\n")
print("Accuracy:", accuracy_score(y_val, nb_preds), "\n")
print(classification_report(y_val, nb_preds))
print("Confusion Matrix:\n", confusion_matrix(y_val, nb_preds))


Naive Bayes Validation Performance:

Accuracy: 0.7877094972067039 

              precision    recall  f1-score   support

         0.0       0.81      0.84      0.82       105
         1.0       0.76      0.72      0.74        74

    accuracy                           0.79       179
   macro avg       0.78      0.78      0.78       179
weighted avg       0.79      0.79      0.79       179

Confusion Matrix:
 [[88 17]
 [21 53]]


In [37]:
# kNN
print("KNN Validation Performance (k=5):\n")
print("Accuracy:", accuracy_score(y_val, knn_preds), "\n")
print(classification_report(y_val, knn_preds))
print("Confusion Matrix:\n", confusion_matrix(y_val, knn_preds))

KNN Validation Performance (k=5):



ValueError: Found input variables with inconsistent numbers of samples: [179, 418]